<a href="https://colab.research.google.com/github/Kratosgado/audio-steganography/blob/main/steg-ai/core_modules/Final_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install stable-baselines3 shimmy --quiet

In [2]:
import os
!git clone https://github.com/librosa/data.git ./audio_data
# get audio files from librose
audios_path = "./audio_data/audio"
simple_audio_files = [os.path.join(audios_path, f) for f in os.listdir(audios_path) if f.endswith(".ogg")]
complex_audio_files = [os.path.join(audios_path, f) for f in os.listdir(audios_path) if f.endswith(".ogg")]

Cloning into './audio_data'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 156 (delta 61), reused 125 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (156/156), 14.78 MiB | 10.13 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import librosa
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
from torch.distributions import Normal
from scipy.signal import get_window
from scipy.fft import dct, idct
import scipy.io.wavfile as wf
import scipy
import requests
import torch.nn.functional as F
from collections import deque
import gymnasium as gym
from gymnasium import spaces
import random
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.callbacks import BaseCallback

In [4]:
# warnings.filterwarnings("ignore")
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
FRAME_SIZE = 1024 #2048
HOP_LENGTH = 256 #512
NON_CRITICAL_PERCENT = 0.1 # Modify bottom 10% of coeffs by magnitude
STATE_DIM = 1024
SAMPLE_RATE = 22050
N_MELS = 128

Using device: cuda


# --- UTILITY FUNCTIONS ---

In [5]:
def download_file(url, local_filename): # gemini
  """Downloads a file if it doesn't already exist."""
  if not os.path.exists(local_filename):
    print(f"Downloading {local_filename}...")
    with requests.get(url, stream=True) as r:
      r.raise_for_status()
      with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
          f.write(chunk)
    print(f"{local_filename} downloaded successfully.")
  else:
    print(f"{local_filename} already exists.")

NORM_NUM = 32768
def textToBits( message):
  return ''.join(format(ord(ch), '08b') for ch in message)


def bitToChar(bits):
  return chr(int(bits, 2))

def textFromBits(bits):
  text = ''
  for bit in range(0, len(bits), 8):
    text += bitToChar(bits[bit:bit + 8])
  return text

# Function to generate a random message (for the RL environment)
def generate_random_message(length=50):
    """Generates a random string message."""
    import string
    letters = string.ascii_letters + string.digits + string.punctuation + " "
    return ''.join(random.choice(letters) for i in range(length))
def extract_state(audio_segment, message_context, sr: int | float=44100):
  # Audio features
  mfcc = librosa.feature.mfcc(y=audio_segment, sr=sr)
  spectral_centroid = librosa.feature.spectral_centroid(y=audio_segment, sr=sr)

  # Temporal features
  rms = librosa.feature.rms(y=audio_segment)
  zcr = librosa.feature.zero_crossing_rate(y=audio_segment)

  # Normalize and flatten features
  return np.concatenate([mfcc.mean(axis=1),spectral_centroid.flatten(), rms.flatten(),zcr.flatten(),message_context ])

def calculate_reward(stego_audio, original_audio, message, detection_prob):
  # imperceptibility (audio quality)
  psnr = 10 * np.log10(np.max(original_audio ** 2) / np.mean((original_audio - stego_audio) ** 2))

  # message integrity
  extracted = extract_message(stego_audio)
  extracted = ''
  ber = np.mean(extracted != message)

  # capacity utilization
  capacity_util = len(message) / max_possible_capacity

  # steganalysis evasion
  evasion_bonus = 1 - detection_prob

  # combined reward
  reward = (
      0.4 * psnr / 30 + # normalized PSNR (target ~30dB)
      0.3 * (1 - ber) + # BER penalty (message accuracy)
      0.2 * capacity_util + # capacity utilization bonus
      0.1 * evasion_bonus # steganalysis evasion bonus
  )
  return reward, {'psnr': psnr, 'ber': ber, 'capacity_util': capacity_util, 'evasion_bonus': evasion_bonus}

class CustomLoggingCallback(BaseCallback):
  """ A custom callback that logs additional information from the environment. """
  def __init__(self, verbose=0):
    super(CustomLoggingCallback, self).__init__(verbose)

  def _on_step(self) -> bool:
    """This method is called after each step in the environment.   """
    # Accessing environment infos from the VecEnv wrapper
    if self.locals.get('infos'):
      for info in self.locals['infos']:
        # Ensure info is a dictionary and contains the required keys
        if isinstance(info, dict) and 'snr' in info and 'detection_prob' in info:
          self.logger.record('rollout/ep_snr', info['snr'])
          self.logger.record('rollout/ep_reward', info['reward'])
          self.logger.record('rollout/ep_extraction_accuracy', info['extraction_accuracy'])
          self.logger.record('rollout/ep_detection_prob', info['detection_prob'])
          self.logger.record('rollout/action', info['action'])
    return True

# --- HYPERPARAMETERS ---

In [6]:
class Config:
  """Class to hold all hyperparameters."""
  #Audio processing
  FRAME_SIZE = 1024 #2048
  HOP_LENGTH = 256 #512
  NON_CRITICAL_PERCENT = 0.1 # Modify bottom 10% of coeffs by magnitude
  STATE_DIM = 1024
  SAMPLE_RATE = 22050
  N_MELS = 128

  # RL Training
  EPISODES = 1000
  LEARNING_RATE_ACTOR = 3e-4
  LEARNING_RATE_CRITIC = 3e-4
  GAMMA = 0.99 # Discount factor
  GAE_LAMBDA = 0.95 # Lambda for Generalized Advantage Estimation
  PPO_EPSILON = 0.2 # Epsilon for clipping in PPO
  PPO_EPOCHS = 10 # Number of epochs for PPO update
  BATCH_SIZE = 64

  # Environment Network Pre-training
  PRETRAIN_EPOCHS = 10
  PRE_TRAIN_LR = 1e-3
  PRE_TRAIN_BATCH_SIZE = 32

  # Reward weights
  SNR_WEIGHTS = 0.02 # Weight for SNR in reward. Tuned to be on a similar scale to detection prob.
  DETECTION_WEIGHT = 1.0 # Weight for detection probability in reward.
  IMPERCEPTIBILITY_WEIGHT = 0.3
  UNDETECTABILITY_WEIGHT = 0.4
  EXTRACTION_ACCURACY_WEIGHT = 0.3
cfg = Config()

# --- MODULE 1: AUDIO PREPROCESSOR ---

In [7]:
class AudioPreprocessor:
  """Handles audio loading, MDCT, and inverse MDCT."""
  def __init__(self, audio_path= None, audio_data=None, frame_size=1024, hop_length=256, sr=22050):
    if audio_path:
      self.audio, self.sr = librosa.load(audio_path, sr=sr, mono=True)
    elif audio_data is not None:
      self.audio = audio_data
      self.sr = sr
    else:
      raise ValueError("Either audio_path or audio_data must be provided")

    # Normalize audio
    self.audio = self.audio / np.max(np.abs(self.audio))
    self.frame_size = frame_size
    self.hop_length = hop_length
    self.window = get_window('hann', self.frame_size)
    # return self.audio, self.sr

  def load_audio(self, path):
    """Load WAV audio file"""
    audio, _ = librosa.load(path, sr=self.sr)
    return audio

  def stft(self, audio):
    """Compute Short-Time Fourier Transform (STFT)"""
    return librosa.stft(audio, n_fft=self.frame_size, hop_length=self.hop_length)

  def istft(self, stft_matrix):
    """Compute Inverse Short-Time Fourier Transform (ISTFT)"""
    return librosa.istft(stft_matrix, hop_length=self.hop_length)

  def compute_mdct(self):
    """Compute Modified Discrete Cosine Transform (MDCT) using STFT and DCT from librosa"""
    stft = self.stft(self.audio)
    magnitudes = np.abs(stft)
    phases = np.angle(stft)
    return magnitudes, phases

  def get_non_critical_coeffs(self, magnitudes, percentile=10):
    """Identify non-critical coefficients (lowest magnitude)"""
    threshold = np.percentile(np.abs(magnitudes.flatten()), percentile)
    mask = np.abs(magnitudes) < threshold
    return mask

  def reconstruct_audio(self, magnitudes, phases):
    """Reconstruct audio from magnitude/stft matrix and phase/non_critical_coeffs"""
    stft = magnitudes * np.exp(1j * phases)
    reconstructed_audio = self.istft(stft)
    return reconstructed_audio

  def plot_spectrogram(self, title):
    """Visualize audio spectrogram"""
    plt.figure(figsize=(10, 4))
    S = librosa.amplitude_to_db(np.abs(librosa.stft(self.audio)), ref=np.max)
    librosa.display.specshow(S, sr=self.sr, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.tight_layout()
    plt.show()

  def save_audio(self, audio: np.ndarray, sr: int, path: str):
    wf.write(path,sr, (audio * NORM_NUM).astype(np.int16))

# --- MODULE 5: EMBEDDING/EXTRACTION MODULE ---
Main Sign encoding class

In [8]:
from abc import ABC, abstractmethod

class EmbeddingModule(ABC):
  def __init__(self):
    pass

  @abstractmethod
  def embed(self, *args, **kwargs):
    pass

  @abstractmethod
  def extract(self, *args, **kwargs) -> str:
    pass

class SignEncoding(EmbeddingModule):
  """Embeds and extracts messages from MDCT coefficients."""
  def __init__(self, steganalysis_net=None, method='mdct'):
        self.steganalysis_net = steganalysis_net
        if self.steganalysis_net:
            self.steganalysis_net.to(device)
            self.steganalysis_net.eval()

  def embed(self, magnitudes, mask, bin_message: list[int], alpha=0.01):
    """Embed message using sign encoding in non-critical coefficients"""
    # Apply mask to get non-critical coefficients
    coeffs = magnitudes.copy()
    non_critical = coeffs[mask]

    # Ensure we have enough coefficients for the message
    if len(non_critical) < len(bin_message):
        raise ValueError("Message too long for available non-critical coefficients")

    # Embed message using sign encoding
    for i, bit in enumerate(bin_message):
        sign = 1 if bit == 1 else -1
        non_critical[i] = sign * np.abs(non_critical[i]) * (1 + alpha)

    # Update coefficients
    coeffs[mask] = non_critical
    return coeffs

  def extract(self, magnitudes, mask, message_length):
    """Extract message from non-critical coefficients"""
    # Apply mask to get non-critical coefficients
    non_critical = magnitudes[mask]

    # Extract message from sign
    bin_message = []
    for i in range(message_length * 8):
        sign = 1 if non_critical[i] >= 0 else -1
        bit = 1 if sign > 0 else 0
        bin_message.append(str(bit))

    # Convert binary to string
    bin_str = ''.join(bin_message)
    chars = [chr(int(bin_str[i:i+8], 2)) for i in range(0, len(bin_str), 8)]
    return ''.join(chars)

  def detect(self, audio):
    """Compute detection probability using steganalysis network"""
    if not self.steganalysis_net:
      raise ValueError("Steganalysis network not initialized")

    # Convert to tensor
    audio_tensor = torch.as_tensor(audio, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

    with torch.no_grad():
      prob = self.steganalysis_net(audio_tensor).item()
    return prob

### Spread Spectrum

In [9]:
class SpreadSpectrum(EmbeddingModule):
  def __init__(self, carrier_freq=10000, chip_rate=100, snr=20):
    """
    Initialize the steganography system

    Parameters:
      carrier_freq (int): Carrier frequency in Hz for embedding
      chip_rate (int): How many samples per bit (spreading factor)
      snr (int): Desired Signal to noise ratio in dB for embedding
    """
    self.carrier_freq = carrier_freq
    self.chip_rate = chip_rate
    self.snr_db = snr
    self.sample_rate = 4100 # standard audio sample rate

    # Parameters for Gold sequence generation (example values)
    self.taps1 = [5, 2]
    self.taps2 = [5, 4, 2, 1] # Must be a preferred pair with taps1
    self.seed1 = 0b11111
    self.seed2 = 0b10101


  def _generate_m_sequence(self, taps, length, initial_state):
      """Generate an m-sequence."""
      lfsr = initial_state
      seq = np.zeros(length)
      # mask = sum(1 << (t - 1) for t in taps) # Mask is not used in this implementation

      for i in range(length):
          seq[i] = 1 if (lfsr & 1) else -1  # Convert to bipolar (-1, 1)
          feedback = 0
          for tap in taps:
              feedback ^= (lfsr >> (tap - 1)) & 1
          # Adjust the shift based on the number of bits in the initial state
          lfsr = (lfsr >> 1) | (feedback << (len(bin(initial_state)) - 3)) # Assuming initial_state is not 0

      return seq

  def _generate_spreading_code(self, length):
      """Generate a Gold sequence."""
      # The lengths of the m-sequences must be the same
      # and derived from the same primitive polynomial.
      # The taps define the primitive polynomials.
      # The seeds are the initial states of the LFSRs.

      m_seq1 = self._generate_m_sequence(self.taps1, length, self.seed1)
      m_seq2 = self._generate_m_sequence(self.taps2, length, self.seed2)

      # XOR the two m-sequences
      gold_seq = m_seq1 * m_seq2 # For bipolar sequences, XOR is multiplication

      return gold_seq


  def _calculate_required_bandwidth(self, message_length):
    """Calculate the required bandwidth based on message length."""
    # each bit is spread over chip_rate samples
    return message_length * self.chip_rate

  def embed(self, audio_path, message, output_path):
    """
    Embed a message into an audio file.

    Parameters:
      audio_path (str): Path to the audio file.
      message (str): The message to be embedded.
      output_path (str): Path to save the embedded audio file.
    """
    # load audio file
    original_audio, sr = librosa.load(audio_path, sr=self.sample_rate)

    # convert message to binary
    message_bin = textToBits(message)
    message_bin = np.array([int(b) for b in message_bin], dtype=np.float32)
    message_bin = message_bin * 2 - 1 # convert to bipoloar (-1, 1)

    # generate spreading codes
    code_length  = len(message_bin) * self.chip_rate
    # spreading_code = self._generate_spreading_code(code_length, seed=42) # Old simplified code
    spreading_code = self._generate_spreading_code(code_length)


    # create the spread message signal
    spread_message = np.repeat(message_bin, self.chip_rate) * spreading_code

    # create carrier signal (sine wave at carrier frequency)
    t = np.arange(len(spread_message)) / sr
    carrier = np.sin(2 * np.pi * self.carrier_freq * t)

    # modulate the message onto the carrier
    modulated = spread_message * carrier

    # adjust the signal power based on desired snr
    signal_power = np.var(original_audio)
    message_power = np.var(modulated)
    desired_message_power = signal_power/ (10 ** (self.snr_db / 10))
    scaling_factor = np.sqrt(desired_message_power / message_power)
    modulated = modulated * scaling_factor

    # pad or truncate the modulated signal to match audio length
    if len(modulated) < len(original_audio):
      modulated = np.pad(modulated, (0, len(original_audio) - len(modulated)), 'constant')
    else:
      modulated = modulated[:len(original_audio)]

    # embed the message into the audio
    stego_audio = original_audio + modulated

    sf.write(output_path, stego_audio, sr)  # save the stego audio
    return stego_audio

  def extract(self, stego_path, message_length, original_audio_path=None):
    """
        Extract a hidden message from a stego audio file.

        Parameters:
            stego_path (str): Path to the stego audio file
            message_length (int): Length of the hidden message in bits
            original_audio_path (str): Optional path to original audio for comparison
        """
    # load stego audio
    y_stego, sr = librosa.load(stego_path, sr=self.sample_rate)

    # if original audio is provided, subtract it to get just the message
    if original_audio_path:
      y_original, _ = librosa.load(original_audio_path, sr=self.sample_rate)
      y_diff = y_stego - y_original
    else:
      y_diff = y_stego

    # generate the same spreading code used in embedding
    code_length = message_length * 8 * self.chip_rate # 8 bits per character
    # spreading_code = self._generate_spreading_code(code_length, seed=42) # Old simplified code
    spreading_code = self._generate_spreading_code(code_length)


    # create carrier signal
    t = np.arange(len(spreading_code)) / sr
    carrier = np.sin(2 * np.pi * self.carrier_freq * t)

    # pad or truncate the carrier to match the difference signal
    if len(carrier) < len(y_diff):
      carrier = np.pad(carrier, (0, len(y_diff) - len(carrier)), 'constant')
    else:
      carrier = carrier[:len(y_diff)]

    # demodulate the signal
    demodulated = y_diff * carrier

    # correlate with spreading code to extract bits
    extracted_bits = ''
    for i in range(message_length * 8):
      start = i * self.chip_rate
      end = start + self.chip_rate
      if end > len(demodulated):
        break
      segment = demodulated[start:end]
      code_segment = spreading_code[start:end]

      # calculate correlation
      correlation = np.sum(segment * code_segment)
      extracted_bits += ('1' if correlation > 0 else '0')

    # convert bits to string
    return extracted_bits

# --- MODULE 2: RL ACTOR-CRITIC NETWORKS ---

In [10]:
class PolicyNetwork(nn.Module):
  """Actor Network: Decides on the modification scale."""
  def __init__(self, input_dim, hidden_dim=256):
    super(PolicyNetwork, self).__init__()
    self.fc = nn.Sequential(
        nn.Linear(input_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, hidden_dim),
        nn.ReLU()
    )
    self.mean_layer = nn.Linear(hidden_dim, 1)
    self.log_std_layer = nn.Linear(hidden_dim, 1)

  def forward(self, x):
    features = self.fc(x)
    mean = self.mean_layer(features)
    log_std = self.log_std_layer(features)
    log_std = torch.clamp(log_std, min=-20, max=2)  # Constrain for stability
    return mean, log_std

  def sample_action(self, state):
    """Sample action from policy distribution"""
    state_tensor = torch.as_tensor(state, dtype=torch.float32).to(device)
    mean, log_std = self.forward(state_tensor)
    std = torch.exp(log_std)
    normal_dist = torch.distributions.Normal(mean, std)
    action = normal_dist.sample()
    return action.cpu().detach().numpy().flatten()

# 7. Feature Extractor for PPO
class CustomFeatureExtractor(BaseFeaturesExtractor):
  def __init__(self, observation_space: spaces.Box, features_dim = 128):
    super().__init__(observation_space, features_dim)
    self.net = nn.Sequential(
        nn.Linear(observation_space.shape[0], 256),
        nn.ReLU(),
        nn.Linear(256, features_dim),
        nn.ReLU()
    )

  def forward(self, observations: torch.Tensor) -> torch.Tensor:
    return self.net(observations)

# -- Action and Observation Spaces --

In [11]:
class ActionSpace(gym.Space):
  def __init__(self):
    # Define the bounds for continuous actions
    self.continuous_bounds = np.array([
        [5000, 15000], # carrier frequency
        [50, 200], # chip rate
        [10, 30] # snr
    ], dtype=np.float32) # Specify dtype as float32

    # Define the number of discrete options
    self.discrete_options = 3 # band selection options

    # Define the action space as a Tuple of Box and Discrete spaces
    self.spaces = (
        gym.spaces.Box(low=self.continuous_bounds[:, 0], high=self.continuous_bounds[:, 1], dtype=np.float32),
        gym.spaces.Discrete(self.discrete_options)
    )

    # Initialize the parent class
    super().__init__(None, None) # Bounds and dtype are defined by the spaces

  def sample(self):
    # Sample from the continuous and discrete spaces
    cont = self.spaces[0].sample()
    disc = self.spaces[1].sample()
    # Return the sample as a tuple, matching the structure of the spaces
    return (cont, disc)

  def contains(self, x):
    # Check if an action is within the space
    if not isinstance(x, tuple) or len(x) != 2:
        return False
    return self.spaces[0].contains(x[0]) and self.spaces[1].contains(x[1])

  def __repr__(self):
        return "SteganographyActionSpace"

  def __eq__(self, other):
        return isinstance(other, ActionSpace)

# --- MODULE 3: ENVIRONMENT NETWORK (STEGANALYZER) ---

In [12]:
class SteganalysisCNN(nn.Module):
  """A 1D CNN that acts as a steganalysis tool."""
  def __init__(self, input_channels=1):
    super(SteganalysisCNN, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv1d(input_channels, 32, kernel_size=5, stride=2, padding=2),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=2),
        nn.Conv1d(32, 64, kernel_size=5, stride=2, padding=2),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=2),
        nn.Flatten()
    )

    # Calculate output dimension after convolutions
    test_input = torch.randn(1, input_channels, 500)
    conv_output_dim = self.conv(test_input).shape[-1]

    self.fc = nn.Sequential(
        nn.Linear(conv_output_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    x = self.conv(x)
    return self.fc(x)

 #5. Custom Gym Environment for RL Training
class AudioStegoEnv(gym.Env):
  def __init__(self, audio_path, bin_message: list[int], method="sign-encoding"):
    super(AudioStegoEnv, self).__init__()

    # Initialize audio and message
    self.preprocessor = AudioPreprocessor(audio_path=audio_path)
    self.original_audio = self.preprocessor.audio.copy()
    self.bin_message = bin_message
    self.message_length = len(bin_message) // 8  # Store message length in characters

    # Compute initial features
    magnitudes, _ = self.preprocessor.compute_mdct()
    self.mask = self.preprocessor.get_non_critical_coeffs(magnitudes)

    # Define action and observation spaces
    self.action_space = spaces.Box(low=0.0, high=0.1, shape=(1,), dtype=np.float32)
    self.observation_space = spaces.Box(low=-1.0, high=1.0,
                                        shape=(N_MELS,), dtype=np.float32)
    self.embedder: EmbeddingModule = SignEncoding() if method == "sign-encoding" else SpreadSpectrum()

    # Initialize state
    self.reset()

  def reset(self, seed=None, options=None):
    """Reset environment to initial state"""
    super().reset(seed=seed) # Call the parent class reset with seed

    self.current_audio = self.original_audio.copy()
    self.current_step = 0
    return self._get_obs(), {} # Return observation and info dictionary

  def _get_obs(self):
    """Extract MFCC features as observation"""
    mfcc = librosa.feature.mfcc(y=self.current_audio, sr=SAMPLE_RATE,
                                n_mfcc=N_MELS, n_fft=FRAME_SIZE,
                                hop_length=HOP_LENGTH)
    return np.mean(mfcc, axis=1)

  def step(self, action):
    """Execute one embedding step"""
    alpha = action[0]

    # Compute MDCT
    magnitudes, phases = self.preprocessor.compute_mdct()

    # Embed message
    modified_magnitudes = self.embedder.embed(magnitudes, self.mask, self.bin_message, alpha)

    # Reconstruct audio
    self.current_audio = self.preprocessor.reconstruct_audio(modified_magnitudes, phases)

    # Compute rewards
    snr = self._calculate_snr()
    detection_prob = self._simulate_detection()
    extraction_accuracy = self._calculate_extraction_accuracy(modified_magnitudes)
    reward = self._calculate_reward(snr, detection_prob, extraction_accuracy)

    # Update state
    self.current_step += 1
    done = self.current_step >= 10  # Train for 10 steps
    info = {
        "snr": snr,
        "detection_prob": detection_prob,
        "extraction_accuracy": extraction_accuracy,
        "reward": reward,
        "action": action
    }
    return self._get_obs(), reward, done, False, info

  def _calculate_snr(self):
    """Calculate Signal-to-Noise Ratio"""
    # Ensure both audio arrays have the same length before calculating noise
    min_len = min(len(self.current_audio), len(self.original_audio))
    current_audio_trimmed = self.current_audio[:min_len]
    original_audio_trimmed = self.original_audio[:min_len]

    noise = current_audio_trimmed - original_audio_trimmed
    signal_power = np.mean(original_audio_trimmed ** 2)
    noise_power = np.mean(noise ** 2)

    if noise_power == 0:
        return 100  # High SNR if no noise

    return 10 * np.log10(signal_power / noise_power)

  def _simulate_detection(self):
    """Simulate steganalysis detection"""
    # not real
    snr = self._calculate_snr()
    return 1 / (1 + np.exp(0.5 * (snr - 50)))  # Logistic function

  def _calculate_extraction_accuracy(self, modified_magnitudes):
    """Calculate the accuracy of extracted bits"""
    # Use the same mask as embedding for extraction simulation
    extracted_bits_str = self.embedder.extract(modified_magnitudes, self.mask, self.message_length)
    extracted_bits = [int(bit) for bit in ''.join(format(ord(char), '08b') for char in extracted_bits_str)]

    # Compare with original binary message
    correct_bits = sum(1 for i in range(min(len(extracted_bits), len(self.bin_message))) if extracted_bits[i] == self.bin_message[i])
    total_bits = len(self.bin_message)
    return correct_bits / total_bits if total_bits > 0 else 0.0

  def _calculate_reward(self, snr, detection_prob, extraction_accuracy):
    """Calculate reward balancing SNR, detectability, and extraction accuracy"""
    # Target: SNR > 50 dB, detection_prob < 0.1, extraction_accuracy close to 1.0
    snr_reward = min(snr / 50, 1.0)
    detect_reward = 1.0 - min(detection_prob, 1.0)

    # Weighted combination
    return (
        cfg.SNR_WEIGHTS * snr_reward +
        cfg.DETECTION_WEIGHT * detect_reward +
        cfg.EXTRACTION_ACCURACY_WEIGHT * extraction_accuracy
    )

# -- MAIN FRAMEWORK --

In [13]:
class RLAudioSteganography:
  """Main framework class"""
  def __init__(self, cfg: Config) -> None:
    self.cfg = cfg
    self.embedded_mask = None # Store the mask used during embedding
    self.original_magnitudes = None # Store original magnitudes

  def Initialize_components(self, audio_path, method="sign-encoding"):
    """Initialize components"""
    self.preprocessor = AudioPreprocessor(audio_path=audio_path)
    self.original_magnitudes, self.phases = self.preprocessor.compute_mdct()
    self.mask = self.preprocessor.get_non_critical_coeffs(self.original_magnitudes)
    self.steganalysis_net = SteganalysisCNN()
    # self.steganalysis_net = SteganalysisCNN(input_channels=len(self.preprocessor.audio))
    self.steganalysis_net.to(device)
    self.embedder: EmbeddingModule = SignEncoding() if method == "sign-encoding" else SpreadSpectrum()

  def string_to_bits(self, message):
    """Convert a string to a sequence of bits."""
    return [int(bit) for bit in ''.join(format(ord(char), '08b') for char in message)]

  def bits_to_string(self, bits):
    """Convert a sequence of bits to a string."""
    text = ''
    for bit in range(0, len(bits), 8):
      if bit + 8 <= len(bits):
        byte = ''.join(str(bit) for bit in bits[bit:bit + 8])
        text += chr(int(byte, 2))
    return text

  def compute_reward(self, detection_prob, snr, extraction_accuracy):
    """Compute reward for RL training"""
    undetectability_reward = 1.0 - detection_prob
    imperceptibility_reward = min(snr/ 50.0, 1.0) # normalize to [0, 1]

    # weighted sum
    total_reward = (
        self.cfg.UNDETECTABILITY_WEIGHT * undetectability_reward +
        self.cfg.IMPERCEPTIBILITY_WEIGHT * imperceptibility_reward +
        self.cfg.EXTRACTION_ACCURACY_WEIGHT * extraction_accuracy
    )
    return total_reward

  def train_ppo(self, audio_path, message, total_timesteps=10000)-> PPO:
    """Train PPO agent for audio steganography"""
    env = make_vec_env(lambda: AudioStegoEnv(audio_path, self.string_to_bits(message)), n_envs=4)

    policy_kwargs = dict(
        features_extractor_class=CustomFeatureExtractor,
        features_extractor_kwargs=dict(features_dim=128),
    )

    model = PPO("MlpPolicy", env, policy_kwargs=policy_kwargs, verbose=1,
                device=device, learning_rate=self.cfg.LEARNING_RATE_ACTOR, n_steps=self.cfg.EPISODES)
    # Instantiate the custom callback
    custom_callback = CustomLoggingCallback()
    model.learn(total_timesteps, custom_callback)

    return model

  def embed_message(self, audio_path, message, output_path, model):
    """Embed a message into an audio file using trained policy"""
    # Re-initialize preprocessor and compute magnitudes/phases/mask for the specific audio being embedded into
    self.preprocessor = AudioPreprocessor(audio_path=audio_path)
    self.original_magnitudes, self.phases = self.preprocessor.compute_mdct()
    self.mask = self.preprocessor.get_non_critical_coeffs(self.original_magnitudes)
    self.embedded_mask = self.mask # Store the mask used for embedding

    # Steganalysis network is initialized in Initialize_components, ensure it's done before embedding
    # If embed_message is called standalone, ensure Initialize_components is called first with the correct audio_path

    env = AudioStegoEnv(audio_path, self.string_to_bits(message)) # Use the correct audio_path and message
    obs, info = env.reset()
    action, _ = model.predict(obs, deterministic=True)
    # Convert message string to bits for embedding
    message_bits = self.string_to_bits(message)
    self.modified_magnitudes = self.embedder.embed(self.original_magnitudes, self.embedded_mask, message_bits, alpha=action[0])
    stego_audio = self.preprocessor.reconstruct_audio(self.modified_magnitudes, self.phases)
    self.preprocessor.save_audio(stego_audio,SAMPLE_RATE, output_path)
    print(f"Stego audio saved to {output_path}")


  def extract_message(self, stego_audio_path, msg_length):
    """Extract a message from a stego audio file"""
    # Load the stego audio
    preprocessor_stego = AudioPreprocessor(audio_path=stego_audio_path)
    magnitudes_stego, phases_stego = preprocessor_stego.compute_mdct()

    # Use the stored mask from embedding for extraction
    if self.embedded_mask is None:
        raise ValueError("Embedding must be performed before extraction to get the mask.")

    # Use the embedder to extract
    # The embedder's extract method requires the magnitudes from the stego audio,
    # the stored mask, and the original message length in characters.
    extracted_message = self.embedder.extract(self.modified_magnitudes, self.embedded_mask, msg_length)
    return extracted_message


  def plot_training_history(self):
    """Plot training metrics"""
    # plt.figure(figsize=(12, 8))
    # plt.subplot(2, 2, 1)
    # plt.plot(self.training_history['rewards'], label='Reward')
    # plt.title('Training Rewards')
    # plt.xlabel('Episode')
    # plt.ylabel('Average Reward')
    # plt.legend()
    # plt.tight_layout()
    # plt.show()

# --- MAIN EXECUTION SCRIPT ---

In [14]:
if __name__ == "__main__":
    # Initialize the framework
    cfg = Config()
    framework = RLAudioSteganography(cfg)

    # --- Training Phase ---
    # Load sample audio for training
    training_audio_path = librosa.ex('trumpet')
    message_to_embed = "There are things that we do not wish to know. and this is a secret I want to send to you okay"

    # Initialize components with the training audio
    framework.Initialize_components(training_audio_path)

    # Train PPO agent
    print("Training PPO agent...")
    model = framework.train_ppo(training_audio_path, message_to_embed, total_timesteps=100)

    # Save the trained model
    model_save_path = "ppo_audio_stego_model"
    model.save(model_save_path)
    print(f"Trained model saved to {model_save_path}")

    # --- Embedding with the Trained Model on a New Audio ---
    print("\nEmbedding message in a new audio file using the trained model...")

    # Load the saved model
    loaded_model = PPO.load(model_save_path)
    print(f"Loaded model from {model_save_path}")

    # Define a new audio file and output path
    new_audio_path = librosa.ex('trumpet', hq=True) # Using a different trumpet example
    new_output_path = "stego_new_audio.wav"

    # Initialize components with the *new* audio
    framework.Initialize_components(new_audio_path)

    # Embed message using the loaded model
    framework.embed_message(new_audio_path, message_to_embed, new_output_path, loaded_model)

    # Extract message from the new stego audio
    extracted_message = framework.extract_message(new_output_path, len(message_to_embed))
    print(f"\nOriginal Message: {message_to_embed}")
    print(f"Extracted Message from new audio: {extracted_message}")

    # --- Evaluation (Optional) ---
    # You can add evaluation steps here similar to the commented out code
    # to check SNR and detection probability on the new stego audio.

/usr/local/lib/python3.11/dist-packages/audioread/rawread.py:16: DeprecationWarning: 'aifc' is deprecated and slated for removal in Python 3.13
  import aifc
/usr/local/lib/python3.11/dist-packages/audioread/rawread.py:17: DeprecationWarning: 'audioop' is deprecated and slated for removal in Python 3.13
  import audioop
/usr/local/lib/python3.11/dist-packages/audioread/rawread.py:19: DeprecationWarning: 'sunau' is deprecated and slated for removal in Python 3.13
  import sunau


Training PPO agent...
Using cuda device


/usr/local/lib/python3.11/dist-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 4000`, after every 62 untruncated mini-batches, there will be a truncated mini-batch of size 32
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=1000 and n_envs=4)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


--------------------------------------------
| rollout/                  |              |
|    action                 | [0.1]        |
|    ep_detection_prob      | 3.365486e-17 |
|    ep_extraction_accuracy | 1            |
|    ep_len_mean            | 10           |
|    ep_rew_mean            | 13.2         |
|    ep_reward              | 1.3199999    |
|    ep_snr                 | 125.86075    |
| time/                     |              |
|    fps                    | 17           |
|    iterations             | 1            |
|    time_elapsed           | 223          |
|    total_timesteps        | 4000         |
--------------------------------------------


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Trained model saved to ppo_audio_stego_model

Embedding message in a new audio file using the trained model...
Loaded model from ppo_audio_stego_model
Stego audio saved to stego_new_audio.wav

Original Message: There are things that we do not wish to know. and this is a secret I want to send to you okay
Extracted Message from new audio: There are things that we do not wish to know. and this is a secret I want to send to you okay


In [15]:
cfg = Config()
framework_test = RLAudioSteganography(cfg)

audio = '/content/drive/MyDrive/Colab Notebooks/steganography/input.wav'
message = """
    Embed a message into an audio file.

    Parameters:
      audio_path (str): Path to the audio file.
      message (str): The message to be embedded.
      output_path (str): Path to save the embedded audio file.
    """
output_path = "test.wav"

framework_test.Initialize_components(audio_path=audio)
# Load the saved model
model_test = PPO.load(model_save_path)
print(f"Loaded model from {model_save_path}")
# Embed message using the loaded model
framework_test.embed_message(audio, message, output_path, model_test)

# Extract message from the new stego audio
extracted_message = framework_test.extract_message(output_path, len(message))
print(f"\nOriginal Message: {message}")
print(f"Extracted Message from new audio: {extracted_message}")

/tmp/ipython-input-7-3352637121.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  self.audio, self.sr = librosa.load(audio_path, sr=sr, mono=True)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/steganography/input.wav'

In [ ]:
  # Initialize the steganography system
stego = SpreadSpectrum(carrier_freq=10000, chip_rate=100, snr=20)

# Embed a message
secret_message = "Hello, this is a secret!"
host_audio = "drive/MyDrive/Colab Notebooks/steganography/input.wav"
stego_audio = "stego_audio.wav"

print(f"Embedding message: {secret_message}")
stego.embed(host_audio, secret_message, stego_audio)

# Extract the message
extracted_message = stego.extract(stego_audio, len(secret_message))
print(f"Extracted message: {extracted_message}")